In [ ]:
import numpy as np
import pandas as pd
from math import radians, cos, sin, sqrt, atan2

# 거리 계산 함수 (Haversine 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2) * sin(dlon/2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance
df_districts = pd.read_excel('북구 행정동 중심좌표.xlsx')
df_hospitals = pd.read_csv('대구광역시 북구_병의원_20240731.csv',encoding='cp949')
df_hospitals['위도'] = df_hospitals['위도'].astype(str)
df_hospitals['경도'] = df_hospitals['경도'].astype(str)

# Extract numeric values and convert to float
df_hospitals['위도'] = df_hospitals['위도'].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
df_hospitals['경도'] = df_hospitals['경도'].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
def calculate_infrastructure_score(lat, lon, hospitals, radius=1.0):
    scores = []
    for _, row in hospitals.iterrows():
        dist = haversine(lat, lon, row['위도'], row['경도'])
        if dist <= radius:
            score = 1 / (dist + 0.1)  # 역수 방식의 점수, 0으로 값이 나오는 것을 방지하기 위해 0.1 추가
            scores.append(score)
    return sum(scores)  # 또는 np.mean(scores)로 평균 점수

# 각 행정동에 대해 인프라 점수 계산
df_districts['인프라점수'] = df_districts.apply(
    lambda row: calculate_infrastructure_score(row['위도'], row['경도'], df_hospitals),
    axis=1
)

print(df_districts[['행정구역', '인프라점수']])

     행정구역       인프라점수
0     고성동   58.258214
1     칠성동   74.577400
2     침산동   71.729141
3     산격동  129.833448
4     대현동   73.909985
5     복현동  151.878795
6     검단동    1.603301
7   무태조야동   63.464691
8     관문동    3.952699
9     태전동   72.613523
10    구암동   15.571942
11    관음동   15.325497
12    읍내동    0.000000
13    동천동  192.926778
14    노원동   65.517768
15    국우동    6.612075


In [ ]:
df_districts.to_excel('북구 행정구역 인프라 점수.xlsx', index=False)